# Training Data Generation

In [1]:
import os
import pandas as pd

from tqdm import tqdm
from constants import *

Define main procedures

In [2]:
def prepare_labels():
    """Load the label descriptor file and preprocess it for merging."""

    df_labels = pd.read_csv(os.path.join(LABEL_DIR, LABEL_FILE),
                            index_col=ORBIT_COL,
                            parse_dates=EVENT_COLS)

    # disregard incompletely labeled orbits
    df_labels = df_labels.dropna()

    # add columns for predecessor and successor events
    df_labels["Prev out"] = df_labels["SK outer out"].shift()
    df_labels["Next in"] = df_labels["SK outer in"].shift(-1)

    return df_labels

In [3]:
def prepare_orbit(orbit_id):
    """Load the specified orbit and preprocess it for merging."""

    file = os.path.join(ORBIT_DIR, ORBIT_FILE(orbit_id))
    df_stats.loc[orbit_id, "does_not_exist"] = not os.path.exists(file)

    if not df_stats.loc[orbit_id, "does_not_exist"]:
        df_orbit = pd.read_csv(file, index_col=DATE_COL, parse_dates=True)

        # determine validity criteria for orbit
        df_stats.loc[orbit_id, "total_nan_count"] = (
                df_orbit.isnull().sum().sum()
        )
        df_stats.loc[orbit_id, "missing_entry_count"] = (
                (df_orbit.index[-1] - df_orbit.index[0]).total_seconds() + 1 - len(df_orbit)
        )
        df_stats.loc[orbit_id, "has_special_conditions"] = (
                df_labels.loc[orbit_id, "SK outer in"] <= df_orbit.index[0]
                or df_orbit.index[0] <= df_labels.loc[orbit_id, "Prev out"]
                or df_labels.loc[orbit_id, "Next in"] <= df_orbit.index[-1]
                or df_orbit.index[-1] <= df_labels.loc[orbit_id, "SK outer out"]
        )
        df_stats.loc[orbit_id, "flux_norm_maximum"] = (
                df_orbit[FLUX_COLS].pow(2).sum(axis=1).pow(0.5).max()
        )
        
        if df_stats.loc[orbit_id].iloc[0:-1].sum() == 0:
            df_orbit[ORBIT_COL] = orbit_id  # add orbit id
            return df_orbit
        else:
            return None  # rule out invalid orbit

In [4]:
def assign_labels(df_train, df_labels):
    """Assign labels to the training time series using event boundary labels."""

    # interplanetary magnetic field
    df_train[LABEL_COL] = 0

    for row in tqdm(range(len(df_labels))):
        event = lambda col: df_labels.iloc[row, df_labels.columns.get_loc(col)]

        # bow shock crossings
        df_train.loc[event("SK outer in"):event("SK inner in"), LABEL_COL] = 1
        df_train.loc[event("SK inner out"):event("SK outer out"), LABEL_COL] = 1

        # magnetosheath
        df_train.loc[event("SK inner in"):event("MP outer in"), LABEL_COL] = 2
        df_train.loc[event("MP outer out"):event("SK inner out"), LABEL_COL] = 2

        # magnetopause crossings
        df_train.loc[event("MP outer in"):event("MP inner in"), LABEL_COL] = 3
        df_train.loc[event("MP inner out"):event("MP outer out"), LABEL_COL] = 3

        # magnetosphere
        df_train.loc[event("MP inner in"):event("MP inner out"), LABEL_COL] = 4

Prepare label descriptor file

In [5]:
df_labels = prepare_labels()
print(f"#healthy orbits: {len(df_labels)}")

#healthy orbits: 4019


Set up orbit validity statistics

In [6]:
df_stats = pd.DataFrame(
    index=df_labels.index,
    columns=["does_not_exist",
             "total_nan_count",
             "missing_entry_count",
             "has_special_conditions",
             "flux_norm_maximum"]
)

Combine orbit data into one big frame

In [7]:
df_train = pd.concat(map(prepare_orbit, tqdm(df_labels.index)))
df_train.iloc[43462:43467]

100%|██████████| 4019/4019 [19:36<00:00,  3.42it/s]


,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Y,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT
DATE,,,,,,,,,,,,,,,,,,,,,
2011-03-24 19:46:22,6897.941,5403.502,-15374.343,29.343,-2.064,-9.552,0.299,1.637,1.395,18118.125361,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049779,55.749454,4.915513e+07,0.670984,0,2
2011-03-24 19:46:23,6898.235,5403.903,-15374.070,29.540,-1.713,-8.633,0.809,1.914,2.789,18118.117945,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049796,55.749455,4.915513e+07,0.670984,2,2
2011-03-24 19:46:23,6898.235,5403.903,-15374.070,29.540,-1.713,-8.633,0.809,1.914,2.789,18118.117945,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049796,55.749455,4.915513e+07,0.670984,2,3
2011-03-24 19:46:24,6898.530,5404.304,-15373.797,29.849,-0.977,-10.833,1.065,4.469,3.514,18118.110927,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049746,55.749453,4.915513e+07,0.670984,0,3
2011-03-24 19:46:25,6898.825,5404.705,-15373.523,29.333,-2.752,-12.014,1.250,5.516,5.414,18118.103052,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049724,55.749452,4.915513e+07,0.670984,0,3


Remove extreme outliers

In [8]:
df_max = df_stats["flux_norm_maximum"]
outlier_orbits = df_stats.index[df_max > df_max.mean() + 3 * df_max.std()]
print(list(outlier_orbits))

[386, 1375, 1615, 1729, 1899, 1946, 3713, 3714, 3718, 4068]


In [9]:
df_train = df_train[~df_train[ORBIT_COL].isin(outlier_orbits)]

Save orbit validity statistics

In [10]:
df_stats.to_excel(os.path.join(MERGED_DIR, "orbit_statistics.xlsx"))

Assign labels to the instances of the training data

In [11]:
assign_labels(df_train, df_labels)
df_train.iloc[16605:16610]

100%|██████████| 4019/4019 [2:21:59<00:00,  2.12s/it]  


,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT,LABEL
DATE,,,,,,,,,,,,,,,,,,,,,
2011-03-24 12:18:45,5804.272,7081.276,-2428.740,-44.922,16.665,4.784,4.239,6.375,3.369,9608.230719,...,5.855501e+06,-48.713393,-27.442143,2.227680,55.955593,4.894640e+07,0.633532,0,2,0
2011-03-24 12:18:46,5803.633,7080.696,-2427.421,-37.371,8.136,7.607,2.316,3.672,3.443,9607.017439,...,5.855501e+06,-48.713393,-27.442143,2.227663,55.955593,4.894640e+07,0.633532,0,2,0
2011-03-24 12:18:47,5802.993,7080.117,-2426.101,-34.104,4.656,14.430,1.949,0.970,1.545,9605.804096,...,5.855501e+06,-48.713393,-27.442143,2.227660,55.955593,4.894640e+07,0.633532,0,2,1
2011-03-24 12:18:48,5802.353,7079.538,-2424.782,-37.385,-0.449,23.100,1.797,3.131,4.607,9604.591160,...,5.855501e+06,-48.713393,-27.442143,2.227669,55.955593,4.894640e+07,0.633532,0,2,1
2011-03-24 12:18:49,5801.714,7078.958,-2423.462,-31.174,2.786,35.948,3.817,7.081,2.152,9603.377895,...,5.855501e+06,-48.713393,-27.442143,2.227612,55.955591,4.894640e+07,0.633532,0,2,1


Save the final training dataframe

In [12]:
df_train.to_csv(os.path.join(MERGED_DIR, TRAIN_FILE))

Generate descriptive overall statistics

In [13]:
df_train_description = df_train.describe()
df_train_description.to_excel(os.path.join(MERGED_DIR, "df_train_total_description.xlsx"))
df_train_description

,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT,LABEL
count,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,...,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07,9.912693e+07
mean,1.336537e+02,-1.735532e+02,-7.902264e+03,3.670526e-01,-1.850193e-01,-4.418589e+00,1.061097e+00,1.399393e+00,1.398633e+00,1.051757e+04,...,-1.468158e+06,-6.708391e-02,3.648269e+00,3.042887e-01,4.752801e+01,5.895299e+07,-4.050004e-03,6.361541e-05,1.863071e+03,9.588953e-01
std,3.663897e+03,3.896013e+03,5.243407e+03,4.066517e+01,4.002489e+01,5.644536e+01,1.652641e+00,2.090043e+00,2.095408e+00,3.981771e+03,...,4.869809e+06,3.423781e+01,3.326465e+01,4.137488e+00,7.097806e+00,8.470830e+06,6.861735e-01,2.110413e-02,1.171023e+03,1.468669e+00
min,-1.074348e+04,-1.131478e+04,-1.662732e+04,-3.955740e+02,-5.540960e+02,-5.791980e+02,0.000000e+00,0.000000e+00,0.000000e+00,2.015986e+03,...,-7.654416e+06,-5.848620e+01,-4.661918e+01,-4.894014e+00,3.885789e+01,4.599997e+07,-1.000000e+00,-2.000000e+00,2.000000e+00,0.000000e+00
25%,-2.286245e+03,-2.820123e+03,-1.196775e+04,-1.758300e+01,-1.115100e+01,-8.230000e+00,1.800000e-01,2.160000e-01,2.380000e-01,7.762218e+03,...,-6.257032e+06,-3.268717e+01,-2.756760e+01,-3.702871e+00,4.059049e+01,5.027654e+07,-6.640699e-01,0.000000e+00,8.130000e+02,0.000000e+00
50%,-8.718500e+01,-8.093000e+01,-9.314936e+03,-4.190000e-01,7.800000e-02,5.670000e-01,5.140000e-01,6.970000e-01,7.100000e-01,1.137714e+04,...,-2.151302e+06,9.515335e+00,4.450861e+00,-4.182666e-01,4.609395e+01,6.009997e+07,-9.113506e-03,0.000000e+00,1.800000e+03,0.000000e+00
75%,2.541941e+03,2.522989e+03,-4.028148e+03,1.553500e+01,1.110200e+01,9.935000e+00,1.286000e+00,1.772000e+00,1.749000e+00,1.302924e+04,...,3.342049e+06,3.250964e+01,3.594925e+01,4.235450e+00,5.465855e+01,6.737828e+07,6.696617e-01,0.000000e+00,2.800000e+03,2.000000e+00
max,1.143029e+04,1.111089e+04,3.154425e+03,5.589880e+02,5.206690e+02,3.956680e+02,3.332820e+02,2.960840e+02,2.866980e+02,1.812033e+04,...,6.239501e+06,3.894385e+01,5.089473e+01,7.036499e+00,5.897771e+01,6.981782e+07,9.999954e-01,2.000000e+00,4.093000e+03,4.000000e+00


Generate descriptive label-wise statistics

In [14]:
df_train_description = df_train.groupby([LABEL_COL]).describe()
df_train_description.to_excel(os.path.join(MERGED_DIR, "df_train_label_description.xlsx"))
df_train_description

X_MSO                                                             \
            count         mean          std        min         25%       50%   
LABEL                                                                          
0      64868418.0  1328.709071  3400.127286  -9972.731 -1200.15475  1092.349   
1       3667178.0 -1804.555668  3155.259710 -10142.408 -3668.66075 -2098.407   
2      14346807.0 -2541.306124  3273.380119 -10743.482 -4682.83600 -2921.892   
3       2286557.0 -2237.364815  3367.251966 -10611.648 -4787.90100 -2492.330   
4      13957973.0 -1773.140865  2566.189258 -10289.062 -3773.34000 -1442.585   

                                  Y_MSO              ... EXTREMA       \
             75%        max       count        mean  ...     75%  max   
LABEL                                                ...                
0      3729.9390  11430.286  64868418.0 -364.065138  ...     0.0  2.0   
1        51.3635  10403.462   3667178.0   22.846575  ...     0.0  1.0   
2      -467.9930  10397.185  14346807.0  309.575659  ...     0.0  1.0   
3       569.0210   5104.713   2286557.0  230.131479  ...     0.0  1.0   
4       265.6390   4869.525  13957973.0   97.515342  ...     0.0  1.0   

            ORBIT                                                         \
            count         mean          std  min     25%     50%     75%   
LABEL                                                                      
0      64868418.0  1836.703060  1205.649831  2.0   739.0  1752.0  2827.0   
1       3667178.0  1994.269877  1121.437101  2.0  1075.0  1997.0  2883.0   
2      14346807.0  1889.098537  1109.776997  2.0   972.0  1837.0  2745.0   
3       2286557.0  1935.025817  1127.884413  2.0   989.0  1917.0  2810.0   
4      13957973.0  1912.601005  1080.173290  2.0  1047.0  1871.0  2724.0   

               
          max  
LABEL          
0      4093.0  
1      4093.0  
2      4093.0  
3      4093.0  
4      4093.0  

[5 rows x 232 columns]